# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Jiayi"
# Set the data and output directories
DATA_ROOT = '/Users/legion/Desktop/Courses/IS389/data'   
OUTPUT_ROOT = '/Users/legion/Desktop/Courses/IS389/output'
TRAIT = 'Kidney Papillary Cell Carcinoma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


In [ ]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Melanoma_(SKCM)']

If no match is found, jump directly to GEO in Part 2.2

In [3]:
trait_subdir = "TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.KIRP.sampleMap_KIRP_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.KIRP.sampleMap_HiSeqV2_PANCAN.gz')

In [4]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [5]:
def check_rows_and_columns(dataframe, display=False):
    """
    Get the lists of row names and column names of a dataset, and optionally observe them.
    :param dataframe:
    :param display:
    :return:
    """
    dataframe_rows = dataframe.index.tolist()
    if display:
        print(f"The dataset has {len(dataframe_rows)} rows, such as {dataframe_rows[:20]}")
    dataframe_cols = dataframe.columns.tolist()
    if display:
        print(f"\nThe dataset has {len(dataframe_cols)} columns, such as {dataframe_cols[:20]}")
    return dataframe_rows, dataframe_cols

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'additional_surgery_metastatic_procedure',
 'age_at_initial_pathologic_diagnosis']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [8]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_sur

In [9]:
candidate_age_cols = [ 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = [ 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [10]:
def preview_df(df, n=5):
    return df.head(n).to_dict(orient='list')

In [11]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [53.0, 71.0, 71.0, 67.0, 80.0],
 'days_to_birth': [-19603.0, -26244.0, -26134.0, -24626.0, -29275.0],
 'year_of_initial_pathologic_diagnosis': [2013.0,
  2008.0,
  2008.0,
  2008.0,
  2006.0]}

In [12]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [13]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [14]:
gender_col = 'gender'

In [15]:
def xena_select_clinical_features(clinical_df, trait, age_col=None, gender_col=None):
    feature_list = []
    trait_data = clinical_df.index.to_series().apply(xena_convert_trait).rename(trait)
    feature_list.append(trait_data)
    if age_col:
        age_data = clinical_df[age_col].apply(xena_convert_age).rename("Age")
        feature_list.append(age_data)
    if gender_col:
        gender_data = clinical_df[gender_col].apply(xena_convert_gender).rename("Gender")
        feature_list.append(gender_data)
    selected_clinical_df = pd.concat(feature_list, axis=1)
    return selected_clinical_df

In [16]:
def xena_convert_trait(row_index: str):
    """
    Convert the trait information from Sample IDs to labels depending on the last two digits.
    Tumor types range from 01 - 09, normal types from 10 - 19.
    :param row_index: the index value of a row
    :return: the converted value
    """
    last_two_digits = int(row_index[-2:])

    if 1 <= last_two_digits <= 9:
        return 1
    elif 10 <= last_two_digits <= 19:
        return 0
    else:
        return -1

In [17]:
def xena_convert_age(cell: str):
    """Convert the cell content about age to a numerical value using regular expression
    """
    match = re.search(r'\d+', str(cell))
    if match:
        return int(match.group())
    else:
        return None

In [18]:
def xena_convert_gender(cell: str):
    """Convert the cell content about gender to a binary value
    """
    if isinstance(cell, str):
        cell = cell.lower()

    if cell == "female":
        return 0
    elif cell == "male":
        return 1
    else:
        return None

In [19]:
import re
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [20]:
def normalize_gene_symbols_in_index(gene_df):
    """Normalize the human gene symbols at the index of a dataframe, and replace the index with its normalized version.
    Remove the rows where the index failed to be normalized."""
    normalized_gene_list = normalize_gene_symbols(gene_df.index.tolist())
    assert len(normalized_gene_list) == len(gene_df.index)
    gene_df.index = normalized_gene_list
    gene_df = gene_df[gene_df.index.notnull()]
    return gene_df

In [21]:
def normalize_gene_symbols(gene_symbols, batch_size=1000):
    """Normalize human gene symbols in batches using the 'mygenes' library"""
    mg = mygene.MyGeneInfo()
    normalized_genes = {}

    # Process in batches
    for i in range(0, len(gene_symbols), batch_size):
        batch = gene_symbols[i:i + batch_size]
        results = mg.querymany(batch, scopes='symbol', fields='symbol', species='human')

        # Update the normalized_genes dictionary with results from this batch
        for gene in results:
            normalized_genes[gene['query']] = gene.get('symbol', None)

    # Return the normalized symbols in the same order as the input
    return [normalized_genes.get(symbol) for symbol in gene_symbols]


In [22]:
import mygene

if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

12 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('RPL31P11', 2), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
10 input query terms found dup hits:	[('SUGT1P1', 2), ('PTPRVP', 2), ('SNORA62', 3), ('IFITM4P', 7), ('HLA-DRB6', 2), ('FUNDC2P2', 2), ('
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
11 input query terms found dup hits:	[('PIP5K1P1', 2), ('HBD', 2), ('PPP1R2P1', 9), ('HSD17B7P2', 2), ('RPSAP9', 2), ('SNORD68', 2), ('SN
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
15 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('ESPNP', 2), ('HBBP1
158 input query terms found no hit:	[

In [23]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Kidney Papillary Cell Carcinoma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2K-A9WE-01,1,53.0,1,0.895608,-2.059826,-0.531035,0.126528,-0.345878,1.12319,0.509706,...,-2.090786,0.932918,-0.150778,0.613183,2.243367,1.33611,0.226223,-1.415027,1.572405,0.879267
TCGA-2Z-A9J1-01,1,71.0,1,0.899708,-2.980426,0.049665,0.552628,-0.381678,0.17029,1.180806,...,-2.090786,-0.086682,0.565422,2.307883,0.294467,-1.28139,0.637223,-2.017627,2.026605,-0.408633
TCGA-2Z-A9J2-01,1,71.0,0,0.552108,-2.707826,-0.531035,0.473928,-0.255978,0.14389,-0.407894,...,2.040714,-0.086682,-0.748878,0.650683,-0.094533,-1.28139,-0.826577,-1.702427,1.751805,-0.769733
TCGA-2Z-A9J3-01,1,67.0,1,0.867508,-4.064326,-0.531035,-0.493572,-0.203878,0.83559,1.648106,...,-2.090786,-0.086682,-0.303278,-1.587117,-2.384733,2.91811,0.174223,-2.745727,2.166005,-2.483133
TCGA-2Z-A9J5-01,1,80.0,1,0.783608,-1.563626,0.829965,0.357528,-0.711678,0.34799,0.500906,...,-2.090786,-0.086682,-0.748878,1.423483,1.622367,0.46401,0.503323,-1.287227,1.909005,0.152467


In [24]:
def judge_and_remove_biased_features(df, trait, trait_type):
    assert trait_type in ["binary", "continuous"], f"The trait must be either a binary or a continuous variable!"
    if trait_type == "binary":
        trait_biased = judge_binary_variable_biased(df, trait)
    else:
        trait_biased = judge_continuous_variable_biased(df, trait)
    if trait_biased:
        print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
    else:
        print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")
    if "Age" in df.columns:
        age_biased = judge_continuous_variable_biased(df, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
            df = df.drop(columns='Age')
        else:
            print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
    if "Gender" in df.columns:
        gender_biased = judge_binary_variable_biased(df, 'Gender')
        if gender_biased:
            print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
            df = df.drop(columns='Gender')
        else:
            print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

    return trait_biased, df

In [25]:
def judge_binary_variable_biased(dataframe, col_name, min_proportion=0.1, min_num=5):
    """
    Check if the distribution of a binary variable in the dataset is too biased to be usable for analysis
    :param dataframe:
    :param col_name:
    :param min_proportion:
    :param min_num:
    :return:
    """
    label_counter = dataframe[col_name].value_counts()
    total_samples = len(dataframe)
    rare_label_num = label_counter.min()
    rare_label = label_counter.idxmin()
    rare_label_proportion = rare_label_num / total_samples

    print(
        f"For the feature \'{col_name}\', the least common label is '{rare_label}' with {rare_label_num} occurrences. This represents {rare_label_proportion:.2%} of the dataset.")

    biased = (len(label_counter) < 2) or ((rare_label_proportion < min_proportion) and (rare_label_num < min_num))
    return bool(biased)


In [26]:
def judge_continuous_variable_biased(dataframe, col_name):
    """Check if the distribution of a continuous variable in the dataset is too biased to be usable for analysis.
    As a starting point, we consider it biased if all values are the same. For the next step, maybe ask GPT to judge
    based on quartile statistics combined with its common sense knowledge about this feature.
    """
    quartiles = dataframe[col_name].quantile([0.25, 0.5, 0.75])
    min_value = dataframe[col_name].min()
    max_value = dataframe[col_name].max()

    # Printing quartile information
    print(f"Quartiles for '{col_name}':")
    print(f"  25%: {quartiles[0.25]}")
    print(f"  50% (Median): {quartiles[0.5]}")
    print(f"  75%: {quartiles[0.75]}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    biased = min_value == max_value

    return bool(biased)

In [27]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

The merged dataset contains 320 samples.
For the feature 'Kidney Papillary Cell Carcinoma', the least common label is '0' with 32 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Kidney Papillary Cell Carcinoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 61.5
  75%: 71.0
Min: 28.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 86 occurrences. This represents 26.88% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [28]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [29]:
from typing import Callable, Optional, List, Tuple, Union, Any

In [30]:
def save_cohort_info(cohort: str, info_path: str, is_available: bool, is_biased: Optional[bool] = None,
                     df: Optional[pd.DataFrame] = None, note: str = '') -> None:
    """
    Add or update information about the usability and quality of a dataset for statistical analysis.

    Parameters:
    cohort (str): A unique identifier for the dataset.
    info_path (str): File path to the JSON file where records are stored.
    is_available (bool): Indicates whether both the genetic data and trait data are available in the dataset, and can be
     preprocessed into a dataframe.
    is_biased (bool, optional): Indicates whether the dataset is too biased to be usable.
        Required if `is_available` is True.
    df (pandas.DataFrame, optional): The preprocessed dataset. Required if `is_available` is True.
    note (str, optional): Additional notes about the dataset.

    Returns:
    None: The function does not return a value but updates or creates a record in the specified JSON file.
    """
    if is_available:
        assert (df is not None) and (is_biased is not None), "'df' and 'is_biased' should be provided if this cohort " \
                                                             "is relevant."
    is_usable = is_available and (not is_biased)
    new_record = {"is_usable": is_usable,
                  "is_available": is_available,
                  "is_biased": is_biased if is_available else None,
                  "has_age": "Age" in df.columns if is_available else None,
                  "has_gender": "Gender" in df.columns if is_available else None,
                  "sample_size": len(df) if is_available else None,
                  "note": note}
    
    if not os.path.exists(info_path):
        with open(info_path, 'w') as file:
            json.dump({}, file)
        print(f"A new JSON file was created at: {info_path}")

    with open(info_path, "r") as file:
        records = json.load(file)
    records[cohort] = new_record

    temp_path = info_path + ".tmp"
    try:
        with open(temp_path, 'w') as file:
            json.dump(records, file)
        os.replace(temp_path, info_path)

    except Exception as e:
        print(f"An error occurred: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise

In [31]:
import json

save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: /Users/legion/Desktop/Courses/IS389/output\Jiayi\Kidney-Papillary-Cell-Carcinoma\cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [33]:
dataset = 'GEO'
trait_subdir = "Kidney-Papillary-Cell-Carcinoma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE144082',
 'GSE19949',
 'GSE26574',
 'GSE32474',
 'GSE36133',
 'GSE36138',
 'GSE36139',
 'GSE40911',
 'GSE40912',
 'GSE40914',
 'GSE41282',
 'GSE42977',
 'GSE48352',
 'GSE48568',
 'GSE57162',
 'GSE63003',
 'GSE68606',
 'GSE68950',
 'GSE85258',
 'GSE95425']

Repeat the below steps for all the accession numbers

In [34]:
def get_relevant_filepaths(cohort_dir):
    """Find the file paths of a SOFT file and a matrix file from the given data directory of a cohort.
    If there are multiple SOFT files or matrix files, simply choose the first one. May be replaced by better
    strategies later.
    """
    files = os.listdir(cohort_dir)
    soft_files = [f for f in files if 'soft' in f.lower()]
    matrix_files = [f for f in files if 'matrix' in f.lower()]
    assert len(soft_files) > 0 and len(matrix_files) > 0
    soft_file_path = os.path.join(cohort_dir, soft_files[0])
    matrix_file_path = os.path.join(cohort_dir, matrix_files[0])

    return soft_file_path, matrix_file_path

In [35]:
# Finished: Biased
cohort = accession_num = "GSE42977"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE42977\\GSE42977_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE42977\\GSE42977_series_matrix.txt.gz')

In [49]:
# Stop: No gene mapping
cohort = accession_num = "GSE48352"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE48352\\GSE48352_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE48352\\GSE48352_series_matrix.txt.gz')

In [56]:
# Finished: Biased
cohort = accession_num = "GSE85258"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE85258\\GSE85258_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE85258\\GSE85258_series_matrix.txt.gz')

In [69]:
# Stop: No obvious traits
cohort = accession_num = "GSE48568"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE48568\\GSE48568_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE48568\\GSE48568_series_matrix.txt.gz')

In [72]:
# Stop: No gene mapping
cohort = accession_num = "GSE41282"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE41282\\GSE41282_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE41282\\GSE41282_series_matrix.txt.gz')

In [80]:
# Finished
cohort = accession_num = "GSE26574"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE26574\\GSE26574_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE26574\\GSE26574_series_matrix.txt.gz')

In [92]:
# Stop: No obvious traits
cohort = accession_num = "GSE40911"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40911\\GSE40911_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40911\\GSE40911_series_matrix.txt.gz')

In [94]:
# Stop: No obvious traits
cohort = accession_num = "GSE19949"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE19949\\GSE19949_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE19949\\GSE19949-GPL3921_series_matrix.txt.gz')

In [96]:
# Stop: No obvious traits
cohort = accession_num = "GSE36133"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36133\\GSE36133_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36133\\GSE36133_series_matrix.txt.gz')

In [101]:
# Finished: Biased
cohort = accession_num = "GSE95425"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE95425\\GSE95425_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE95425\\GSE95425_series_matrix.txt.gz')

In [104]:
# Finished: Biased
cohort = accession_num = "GSE144082"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE144082\\GSE144082_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE144082\\GSE144082_series_matrix.txt.gz')

In [107]:
# Finished
cohort = accession_num = "GSE63003"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE63003\\GSE63003_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE63003\\GSE63003-GPL17964_series_matrix.txt.gz')

In [120]:
# Finished: Biased
cohort = accession_num = "GSE68950"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE68950\\GSE68950_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE68950\\GSE68950_series_matrix.txt.gz')

In [132]:
# Stop: No obvious traits
cohort = accession_num = "GSE40912"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40912\\GSE40912_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40912\\GSE40912_series_matrix.txt.gz')

In [134]:
# MemoryError: Unable to allocate 162. MiB for an array with shape (21254341, 1) and data type float64
cohort = accession_num = "GSE36138"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36138\\GSE36138_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36138\\GSE36138_series_matrix.txt.gz')

In [146]:
# Finished
cohort = accession_num = "GSE57162"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE57162\\GSE57162_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE57162\\GSE57162_series_matrix.txt.gz')

In [158]:
# Stop: No obvious traits
cohort = accession_num = "GSE40914"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40914\\GSE40914_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE40914\\GSE40914-GPL3985_series_matrix.txt.gz')

In [160]:
# Stop: No gene mapping
cohort = accession_num = "GSE36139"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36139\\GSE36139_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE36139\\GSE36139-GPL15308_series_matrix.txt.gz')

In [168]:
# Finished
cohort = accession_num = "GSE32474"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE32474\\GSE32474_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE32474\\GSE32474_series_matrix.txt.gz')

In [180]:
# Finished
cohort = accession_num = "GSE68606"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE68606\\GSE68606_family.soft.gz',
 '/Users/legion/Desktop/Courses/IS389/data\\GEO\\Kidney-Papillary-Cell-Carcinoma\\GSE68606\\GSE68606_series_matrix.txt.gz')

# Prepossessing

In [181]:
from utils import *
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

clinical_data

!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol for all steps of tissue

,!Sample_geo_accession,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
0,!Sample_characteristics_ch1,cell line: H2347,cell line: H2347,cell line: H1437,cell line: HCC78,cell line: HCC78,cell line: H2087,cell line: H2087,cell line: H2009,cell line: H2009,...,cell line: --,cell line: HCC78,cell line: H2347,cell line: H1437,cell line: H2009,cell line: H1437,cell line: H2347,cell line: H2087,cell line: --,cell line: --
1,!Sample_characteristics_ch1,disease state: --,disease state: --,disease state: Leiomyoma,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,...,disease state: Squamous Cell Carcinoma,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --
2,!Sample_characteristics_ch1,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,...,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --
3,!Sample_characteristics_ch1,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,...,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --
4,!Sample_characteristics_ch1,organism part: --,organism part: --,organism part: Uterus,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,...,organism part: Lung,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --
5,!Sample_characteristics_ch1,Sex: --,Sex: --,Sex: female,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,...,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --,Sex: --
6,!Sample_characteristics_ch1,age: --,age: --,age: --,age: --,age: --,age: --,age: --,age: --,age: --,...,age: --,age: --,age: --,age: --,age: --,age: --,age: --,age: --,age: --,age: --
7,!Sample_characteristics_ch1,histology: --,histology: --,histology: Leiomyoma,histology: --,histology: --,histology: --,histology: --,histology: --,histology: --,...,histology: Squamous Cell Carcinoma,histology: --,histology: --,histology: --,histology: --,histology: --,histology: --,histology: --,histology: --,histology: --


In [183]:
tumor_stage_row = clinical_data.iloc[1]
tumor_stage_row.unique()

array(['!Sample_characteristics_ch1', 'disease state: --',
       'disease state: Leiomyoma', 'disease state: Lung_Adenocarcinoma',
       'disease state: Conventional_Clear_Cell_Renal_Cell_Carcinoma',
       'disease state: Squamous Cell Carcinoma',
       'disease state: Stomach Adenocarcinoma',
       'disease state: Large Cell Lymphoma',
       'disease state: Malignant Melanoma',
       'disease state: Recurrent Renal Cell Carcinoma',
       'disease state: Adrenal Cortical Adenoma',
       'disease state: Ovarian Adenocarcinoma',
       'disease state: Gastrointestinal_Stromal_Tumor',
       'disease state: Metastatic Renal Cell Carcinoma',
       'disease state: Non neoplastic liver with cirrosis',
       'disease state: Malignant G1 Stromal Tumor',
       'disease state: melanoma'], dtype=object)

In [185]:
is_gene_availabe = True
trait_row = 1
age_row = None
gender_row = None

trait_type = 'binary'

# Verify and use the functions generated by GPT

# 这个函数将组织类型（tissue type）转换为有关癫痫存在与否的二进制值。
# 它是基于特定的假设，即如果组织类型是“胰腺导管腺癌”（Pancreatic Ductal Adenocarcinoma），则认为癫痫存在（返回1）；否则，认为癫痫不存在（返回0）。
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if (tissue_type == 'disease state: Conventional_Clear_Cell_Renal_Cell_Carcinoma' or tissue_type == 'disease state: Recurrent Renal Cell Carcinoma' or tissue_type == 'disease state: Metastatic Renal Cell Carcinoma'):
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


# def convert_trait(tumor_grade):
#     if (tumor_grade == 'tumor grade: 2' or tumor_grade == 'tumor grade: 3' or tumor_grade == 'tumor grade: 4'):
#         return 1  
#     elif tumor_grade == 'tumor grade: 1':
#         return 0  
#     else:
#         return None

# 这个函数的目的是将年龄的字符串表示转换为一个连续的数值型表示。如果年龄未知（例如，标记为'n.a.'），则返回None。
# 函数尝试从传入的字符串中提取出一个整数作为年龄值。如果字符串的格式不符合预期，导致提取失败，同样返回None。
def convert_age(age_string):
    """
    Convert age string to a continuous numerical value.
    Unknown values are converted to None.
    """
    if age_string.lower() == 'n.a.':
        return None
    try:
        # Extract age as an integer from the string
        age = int(age_string.split(':')[1])
        return age
    except (ValueError, IndexError):
        # In case of any format error or unexpected string structure
        return None


# 这个函数将性别的字符串表示转换为二进制值，其中“female”对应1，“male”对应0。如果性别未知或字符串不符合预期格式，则返回None。
# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(gender_string):
    """
    Convert gender string to a binary value.
    'female' is represented as 1, 'male' as 0.
    Unknown values are converted to None.
    """
    if (gender_string.lower() == 'sex: female' or gender_string.lower() == 'sex: f' or gender_string.lower() == 'gender: female' or gender_string.lower() == 'gender: f'):
        return 1
    elif (gender_string.lower() == 'sex: male' or gender_string.lower() == 'sex: m' or gender_string.lower() == 'gender: male' or gender_string.lower() == 'gender: m') :  # changeed 
        return 0
    else:
        return None

In [186]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

c:\Users\legion\Desktop\TAIS_2024SP\TAIS_2024SP\Jiayi\..\utils.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,GSM1676873,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
Kidney Papillary Cell Carcinoma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [187]:
genetic_data = get_genetic_data(matrix_file)
genetic_data

,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,GSM1676873,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,1932.38000,1032.65000,1282.80000,2688.61000,2189.27000,342.63900,254.99600,2225.93000,2785.70000,1785.58000,...,2212.02000,1105.37000,1429.44000,1034.43000,1963.04000,685.84300,1038.43000,267.58600,854.45600,1051.03000
1053_at,833.80500,1034.23000,647.45200,149.05600,315.93400,155.56400,312.14500,431.22300,325.09200,793.21600,...,199.23700,514.35200,879.66300,1314.85000,625.29200,727.30500,521.02900,205.01300,478.05300,466.55800
117_at,122.25500,59.32650,126.75000,139.89900,98.95540,66.88500,74.15530,131.67200,77.37710,120.16700,...,429.30400,101.45700,91.43080,83.34430,197.26200,115.29900,128.30800,84.83030,110.79500,94.88720
121_at,1134.54000,1058.97000,1107.48000,1712.00000,1175.17000,1004.85000,943.42200,1246.45000,1114.25000,1289.73000,...,1237.05000,848.33100,760.27900,608.74700,750.60200,815.26600,822.66700,513.12200,741.42200,630.62300
1255_g_at,26.65850,41.44600,58.22860,77.86010,70.63870,37.91410,16.55080,106.73700,77.79580,67.82810,...,19.03560,16.15000,8.93552,3.17154,93.98570,33.79360,25.44110,25.68730,32.24910,12.31470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AFFX-ThrX-5_at,7.04814,8.57466,9.11362,12.67890,14.98140,15.68270,5.77798,9.34684,8.36869,8.44274,...,53.56280,7.69069,7.84491,4.28603,24.32590,7.12141,8.55965,18.17220,60.04510,5.14640
AFFX-ThrX-M_at,27.75430,24.09150,31.00100,25.64010,38.78030,16.97470,25.77740,18.24490,25.75680,28.71170,...,71.87460,26.04780,7.86226,5.50425,65.32950,30.76890,7.79044,33.48380,15.24170,23.51990
AFFX-TrpnX-3_at,3.24531,2.72867,20.23790,19.78960,13.26810,13.99090,3.08212,1.03356,29.03380,3.06343,...,2.39710,16.63280,1.82486,5.90471,5.78538,2.84332,2.08798,16.99070,4.29375,2.71848


In [188]:
requires_gene_mapping = True

if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

gene_annotation.columns

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

In [189]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [190]:
genetic_data = normalize_gene_symbols_in_index(genetic_data)

genetic_data

,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,GSM1676873,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
A1CF,411.658000,328.766000,283.038000,441.273000,456.17000,356.891000,370.538000,391.200000,517.63200,330.712000,...,492.9340,252.68000,214.051000,172.401000,360.6670,224.95800,196.6330,423.316000,604.971000,663.523000
A2M,9.933690,29.729700,31.517800,117.207000,66.05450,76.368500,8.343650,77.880200,96.81630,76.772900,...,1938.5400,34.16810,41.075100,78.744600,13.4254,113.81800,80.4261,90.634000,1469.130000,1450.950000
A4GALT,67.969200,42.261300,103.195000,34.880400,37.44720,20.832300,63.035600,70.497400,25.55710,77.666600,...,73.6854,85.04110,131.085000,155.877000,138.5640,170.43300,96.1404,115.264000,131.866000,22.694400
A4GNT,103.083000,19.840200,120.743000,233.940000,132.97200,54.878500,86.099100,86.712800,132.28300,123.381000,...,375.3930,129.99600,115.976000,82.167800,157.9120,148.86000,60.4659,58.103800,47.507900,122.016000
AAAS,253.048000,307.313000,301.599000,54.338500,292.66000,526.452000,285.639000,242.461000,157.34800,88.740300,...,168.3570,331.88900,455.359000,512.032000,378.5850,584.66500,611.1190,200.164000,299.390000,332.704000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDB,7.021830,25.621200,49.129700,56.345100,52.64970,9.202230,57.596900,38.391100,56.36770,7.179120,...,12.7427,82.66240,84.754400,15.316100,56.1309,43.45840,32.1018,13.704900,66.226700,72.470700
ZXDC,245.770000,223.404000,324.696000,408.354000,274.03900,207.566000,41.970100,96.083800,204.60200,277.355000,...,293.5240,105.07400,239.035000,242.894000,112.4240,226.59100,263.2760,15.799600,209.634000,202.095000
ZYX,864.049500,361.164500,368.090000,509.184500,699.28350,711.235500,831.348000,898.541000,529.82600,535.816500,...,956.2580,1056.60850,1895.635000,513.773000,1485.5750,625.40500,2031.3400,1590.970000,872.288000,982.831000
ZZEF1,95.892933,186.514933,183.754067,259.034713,168.38015,311.179743,224.061743,125.673357,188.30636,118.847337,...,187.8084,105.01919,100.009013,136.908567,82.2776,180.19429,119.2507,192.480833,127.771333,95.924913


In [191]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

merged_data

,Kidney Papillary Cell Carcinoma,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAGAB,AAK1,...,ZSWIM1,ZSWIM8,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
GSM1676864,0.0,411.658,9.93369,67.9692,103.0830,253.0480,693.969,52.00740,254.15790,169.690144,...,172.9030,118.234115,1172.070,937.975,4273.62,7.02183,245.7700,864.0495,95.892933,972.634
GSM1676865,0.0,328.766,29.72970,42.2613,19.8402,307.3130,565.707,52.20780,411.08050,138.574204,...,160.9360,164.939945,888.014,810.915,6408.85,25.62120,223.4040,361.1645,186.514933,1135.550
GSM1676866,0.0,283.038,31.51780,103.1950,120.7430,301.5990,614.077,20.14540,301.78795,171.979780,...,149.0170,182.287700,970.310,769.493,7430.47,49.12970,324.6960,368.0900,183.754067,818.291
GSM1676867,0.0,441.273,117.20700,34.8804,233.9400,54.3385,1193.600,26.59360,195.02295,411.052640,...,206.7570,454.626050,910.911,274.856,5882.10,56.34510,408.3540,509.1845,259.034713,659.049
GSM1676868,0.0,456.170,66.05450,37.4472,132.9720,292.6600,869.968,77.00920,291.42050,225.582720,...,188.0690,336.587970,796.490,342.181,4632.58,52.64970,274.0390,699.2835,168.380150,615.557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1676996,0.0,224.958,113.81800,170.4330,148.8600,584.6650,523.556,7.43009,450.45955,134.190732,...,188.5000,199.339950,657.216,1048.500,5762.04,43.45840,226.5910,625.4050,180.194290,707.989
GSM1676997,0.0,196.633,80.42610,96.1404,60.4659,611.1190,447.764,11.25050,342.95000,108.845170,...,205.8840,121.243740,685.352,1352.300,2760.27,32.10180,263.2760,2031.3400,119.250700,713.453
GSM1676998,0.0,423.316,90.63400,115.2640,58.1038,200.1640,860.323,9.11952,255.18895,120.683550,...,89.2204,201.062600,441.399,575.010,1340.71,13.70490,15.7996,1590.9700,192.480833,765.151
GSM1676999,0.0,604.971,1469.13000,131.8660,47.5079,299.3900,596.813,63.87370,294.99555,163.793346,...,135.0100,170.178150,469.864,1394.440,3616.59,66.22670,209.6340,872.2880,127.771333,486.239


In [192]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

The merged dataset contains 137 samples.
For the feature 'Kidney Papillary Cell Carcinoma', the least common label is '1.0' with 11 occurrences. This represents 8.03% of the dataset.
The distribution of the feature 'Kidney Papillary Cell Carcinoma' in this dataset is fine.



False

In [179]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### Initial filtering and clinical data preprocessing

In [ ]:
import gzip

In [ ]:
def line_generator(source, source_type):
    """Generator that yields lines from a file or a string.

    Parameters:
    - source: File path or string content.
    - source_type: 'file' or 'string'.
    """
    if source_type == 'file':
        with gzip.open(source, 'rt') as f:
            for line in f:
                yield line.strip()
    elif source_type == 'string':
        for line in source.split('\n'):
            yield line.strip()
    else:
        raise ValueError("source_type must be 'file' or 'string'")

In [ ]:
from typing import Callable, Optional, List, Tuple, Union, Any
import pandas as pd

In [ ]:
def filter_content_by_prefix(
    source: str,
    prefixes_a: List[str],
    prefixes_b: Optional[List[str]] = None,
    unselect: bool = False,
    source_type: str = 'file',
    return_df_a: bool = True,
    return_df_b: bool = True
) -> Tuple[Union[str, pd.DataFrame], Optional[Union[str, pd.DataFrame]]]:
    """
    Filters rows from a file or a list of strings based on specified prefixes.

    Parameters:
    - source (str): File path or string content to filter.
    - prefixes_a (List[str]): Primary list of prefixes to filter by.
    - prefixes_b (Optional[List[str]]): Optional secondary list of prefixes to filter by.
    - unselect (bool): If True, selects rows that do not start with the specified prefixes.
    - source_type (str): 'file' if source is a file path, 'string' if source is a string of text.
    - return_df_a (bool): If True, returns filtered content for prefixes_a as a pandas DataFrame.
    - return_df_b (bool): If True, and if prefixes_b is provided, returns filtered content for prefixes_b as a pandas DataFrame.

    Returns:
    - Tuple: A tuple where the first element is the filtered content for prefixes_a, and the second element is the filtered content for prefixes_b.
    """
    filtered_lines_a = []
    filtered_lines_b = []
    prefix_set_a = set(prefixes_a)
    if prefixes_b is not None:
        prefix_set_b = set(prefixes_b)

    # Use generator to get lines
    for line in line_generator(source, source_type):
        matched_a = any(line.startswith(prefix) for prefix in prefix_set_a)
        if matched_a != unselect:
            filtered_lines_a.append(line)
        if prefixes_b is not None:
            matched_b = any(line.startswith(prefix) for prefix in prefix_set_b)
            if matched_b != unselect:
                filtered_lines_b.append(line)

    filtered_content_a = '\n'.join(filtered_lines_a)
    if return_df_a:
        filtered_content_a = pd.read_csv(io.StringIO(filtered_content_a), delimiter='\t', low_memory=False, on_bad_lines='skip')
    filtered_content_b = None
    if filtered_lines_b:
        filtered_content_b = '\n'.join(filtered_lines_b)
        if return_df_b:
            filtered_content_b = pd.read_csv(io.StringIO(filtered_content_b), delimiter='\t', low_memory=False, on_bad_lines='skip')

    return filtered_content_a, filtered_content_b



In [ ]:
def get_background_and_clinical_data(file_path,
                                     prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
                                     prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']):
    """Extract from a matrix file the background information about the dataset, and sample characteristics data"""
    background_info, clinical_data = filter_content_by_prefix(file_path, prefixes_a, prefixes_b, unselect=False,
                                                              source_type='file',
                                                              return_df_a=False, return_df_b=True)
    return background_info, clinical_data

In [ ]:
import io
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol for all steps of tissue

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
0,!Sample_characteristics_ch1,cell line: H2347,cell line: H2347,cell line: H1437,cell line: HCC78,cell line: HCC78,cell line: H2087,cell line: H2087,cell line: H2009,cell line: H2009,...,cell line: --,cell line: HCC78,cell line: H2347,cell line: H1437,cell line: H2009,cell line: H1437,cell line: H2347,cell line: H2087,cell line: --,cell line: --
1,!Sample_characteristics_ch1,disease state: --,disease state: --,disease state: Leiomyoma,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,...,disease state: Squamous Cell Carcinoma,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --,disease state: --
2,!Sample_characteristics_ch1,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,...,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --,tumor grading: --
3,!Sample_characteristics_ch1,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,...,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --,disease stage: --
4,!Sample_characteristics_ch1,organism part: --,organism part: --,organism part: Uterus,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,...,organism part: Lung,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --,organism part: --


In [ ]:
def get_unique_values_by_row(dataframe, max_len=30):
    """
    Organize the unique values in each row of the given dataframe, to get a dictionary
    :param dataframe:
    :param max_len:
    :return:
    """
    if '!Sample_geo_accession' in dataframe.columns:
        dataframe = dataframe.drop(columns=['!Sample_geo_accession'])
    unique_values_dict = {}
    for index, row in dataframe.iterrows():
        unique_values = list(row.unique())[:max_len]
        unique_values_dict[index] = unique_values
    return unique_values_dict

In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell line: H2347',
  'cell line: H1437',
  'cell line: HCC78',
  'cell line: H2087',
  'cell line: H2009',
  'cell line: --'],
 1: ['disease state: --',
  'disease state: Leiomyoma',
  'disease state: Lung_Adenocarcinoma',
  'disease state: Conventional_Clear_Cell_Renal_Cell_Carcinoma',
  'disease state: Squamous Cell Carcinoma',
  'disease state: Stomach Adenocarcinoma',
  'disease state: Large Cell Lymphoma',
  'disease state: Malignant Melanoma',
  'disease state: Recurrent Renal Cell Carcinoma',
  'disease state: Adrenal Cortical Adenoma',
  'disease state: Ovarian Adenocarcinoma',
  'disease state: Gastrointestinal_Stromal_Tumor',
  'disease state: Metastatic Renal Cell Carcinoma',
  'disease state: Non neoplastic liver with cirrosis',
  'disease state: Malignant G1 Stromal Tumor',
  'disease state: melanoma'],
 2: ['tumor grading: --',
  'tumor grading: G2/pT1pN0pMX',
  'tumor grading: G3/pT2pN0pMX',
  'tumor grading: G2/pT2pN0pMX',
  'tumor grading: G3/pT4pNXpMX'],
 3: ['d

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Kidney Papillary Cell Carcinoma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Kidney Papillary Cell Carcinoma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
age_row = gender_row = None
convert_age = convert_gender = None

In [ ]:
is_gene_availabe = True
trait_row = 7
age_row = 6
gender_row = 5
trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'histology: Leiomyoma':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


def convert_age(age_string):
    """
    Convert age string to a continuous numerical value.
    Unknown values are converted to None.
    """
    if age_string.lower() == 'n.a.':
        return None
    try:
        # Extract age as an integer from the string
        age = int(age_string.split(': ')[1])
        return age
    except (ValueError, IndexError):
        # In case of any format error or unexpected string structure
        return None


# It sometimes maps 'female' to 0, and sometimes 1. Does it matter?
def convert_gender(gender_string):
    """
    Convert gender string to a binary value.
    'female' is represented as 1, 'male' as 0.
    Unknown values are converted to None.
    """
    if gender_string.lower() == 'sex: male':
        return 0
    elif gender_string.lower() == 'sex: female':
        return 1
    else:
        return None

In [ ]:
def get_feature_data(clinical_df, row_id, feature, convert_fn):
    """select the row corresponding to a feature in the sample characteristics dataframe, and convert the feature into
    a binary or continuous variable"""
    clinical_df = clinical_df.iloc[row_id:row_id + 1].drop(columns=['!Sample_geo_accession'], errors='ignore')
    clinical_df.index = [feature]
    clinical_df = clinical_df.applymap(convert_fn)

    return clinical_df

In [ ]:
def geo_select_clinical_features(clinical_df: pd.DataFrame, trait: str, trait_row: int,
                                 convert_trait: Callable,
                                 age_row: Optional[int] = None,
                                 convert_age: Optional[Callable] = None,
                                 gender_row: Optional[int] = None,
                                 convert_gender: Optional[Callable] = None) -> pd.DataFrame:
    """
    Extracts and processes specific clinical features from a DataFrame representing
    sample characteristics in the GEO database series.

    Parameters:
    - clinical_df (pd.DataFrame): DataFrame containing clinical data.
    - trait (str): The trait of interest.
    - trait_row (int): Row identifier for the trait in the DataFrame.
    - convert_trait (Callable): Function to convert trait data into a desired format.
    - age_row (int, optional): Row identifier for age data. Default is None.
    - convert_age (Callable, optional): Function to convert age data. Default is None.
    - gender_row (int, optional): Row identifier for gender data. Default is None.
    - convert_gender (Callable, optional): Function to convert gender data. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the selected and processed clinical features.
    """
    feature_list = []

    trait_data = get_feature_data(clinical_df, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    if age_row is not None:
        age_data = get_feature_data(clinical_df, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    if gender_row is not None:
        gender_data = get_feature_data(clinical_df, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)

    selected_clinical_df = pd.concat(feature_list, axis=0)
    return selected_clinical_df

In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,GSM1676873,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
Kidney Papillary Cell Carcinoma,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Gender,None,None,1,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Genetic data preprocessing and final filtering

In [ ]:
def get_genetic_data(file_path):
    """Read the gene expression data into a dataframe, and adjust its format"""
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=52, comment='!', delimiter='\t')
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'}).astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data


In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1676864,GSM1676865,GSM1676866,GSM1676867,GSM1676868,GSM1676869,GSM1676870,GSM1676871,GSM1676872,GSM1676873,...,GSM1676991,GSM1676992,GSM1676993,GSM1676994,GSM1676995,GSM1676996,GSM1676997,GSM1676998,GSM1676999,GSM1677000
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,1932.3800,1032.6500,1282.8000,2688.6100,2189.2700,342.6390,254.9960,2225.930,2785.7000,1785.5800,...,2212.0200,1105.370,1429.44000,1034.43000,1963.0400,685.8430,1038.4300,267.5860,854.4560,1051.0300
1053_at,833.8050,1034.2300,647.4520,149.0560,315.9340,155.5640,312.1450,431.223,325.0920,793.2160,...,199.2370,514.352,879.66300,1314.85000,625.2920,727.3050,521.0290,205.0130,478.0530,466.5580
117_at,122.2550,59.3265,126.7500,139.8990,98.9554,66.8850,74.1553,131.672,77.3771,120.1670,...,429.3040,101.457,91.43080,83.34430,197.2620,115.2990,128.3080,84.8303,110.7950,94.8872
121_at,1134.5400,1058.9700,1107.4800,1712.0000,1175.1700,1004.8500,943.4220,1246.450,1114.2500,1289.7300,...,1237.0500,848.331,760.27900,608.74700,750.6020,815.2660,822.6670,513.1220,741.4220,630.6230
1255_g_at,26.6585,41.4460,58.2286,77.8601,70.6387,37.9141,16.5508,106.737,77.7958,67.8281,...,19.0356,16.150,8.93552,3.17154,93.9857,33.7936,25.4411,25.6873,32.2491,12.3147


In [ ]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1598_g_at',
 '160020_at',
 '1729_at',
 '1773_at',
 '177_at',
 '179_at',
 '1861_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at', '179_at', '1861_at']\n"


If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = False

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [ ]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
def normalize_gene_symbols_in_index(gene_df):
    """Normalize the human gene symbols at the index of a dataframe, and replace the index with its normalized version.
    Remove the rows where the index failed to be normalized."""
    normalized_gene_list = normalize_gene_symbols(gene_df.index.tolist())
    assert len(normalized_gene_list) == len(gene_df.index)
    gene_df.index = normalized_gene_list
    gene_df = gene_df[gene_df.index.notnull()]
    return gene_df

In [ ]:
def normalize_gene_symbols(gene_symbols, batch_size=1000):
    """Normalize human gene symbols in batches using the 'mygenes' library"""
    mg = mygene.MyGeneInfo()
    normalized_genes = {}

    # Process in batches
    for i in range(0, len(gene_symbols), batch_size):
        batch = gene_symbols[i:i + batch_size]
        results = mg.querymany(batch, scopes='symbol', fields='symbol', species='human')

        # Update the normalized_genes dictionary with results from this batch
        for gene in results:
            normalized_genes[gene['query']] = gene.get('symbol', None)

    # Return the normalized symbols in the same order as the input
    return [normalized_genes.get(symbol) for symbol in gene_symbols]

In [ ]:
import mygene

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1000 input query terms found no hit:	['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_a
1000 input query terms found no hit:	['201473_at', '201474_s_at', '201475_x_at', '201476_s_at', '201477_s_at', '201478_s_at', '201479_at'
1000 input query terms found no hit:	['202473_x_at', '202474_s_at', '202475_at', '202476_s_at', '202477_s_at', '202478_at', '202479_s_at'
1000 input query terms found no hit:	['203474_at', '203475_at', '203476_at', '203477_at', '203478_at', '203479_s_at', '203480_s_at', '203
1000 input query terms found no hit:	['204474_at', '204475_at', '204476_s_at', '204477_at', '204478_s_at', '204479_at', '204480_s_at', '2
1000 input query terms found no hit:	['205474_at', '205475_at', '205476_at', '205477_s_at', '205478_at', '205479_s_at', '205480_s_at', '2
1000 input query terms found no hit:	['206475_x_at', '206476_s_at', '206477_s_at', '206478_at', '206479_at', '206480_at', '206481_s_at', 
1000 input query terms found no hi

In [ ]:
def geo_merge_clinical_genetic_data(clinical_df, genetic_df):
    """
    Merge the clinical features and gene expression features from two dataframes into one dataframe
    """
    if 'ID' in genetic_df.columns:
        genetic_df = genetic_df.rename(columns={'ID': 'Gene'})
    if 'Gene' in genetic_df.columns:
        genetic_df = genetic_df.set_index('Gene')
    merged_data = pd.concat([clinical_df, genetic_df], axis=0).T.dropna()
    return merged_data


In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 20 samples.


In [ ]:
def judge_and_remove_biased_features(df, trait, trait_type):
    assert trait_type in ["binary", "continuous"], f"The trait must be either a binary or a continuous variable!"
    if trait_type == "binary":
        trait_biased = judge_binary_variable_biased(df, trait)
    else:
        trait_biased = judge_continuous_variable_biased(df, trait)
    if trait_biased:
        print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
    else:
        print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")
    if "Age" in df.columns:
        age_biased = judge_continuous_variable_biased(df, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
            df = df.drop(columns='Age')
        else:
            print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
    if "Gender" in df.columns:
        gender_biased = judge_binary_variable_biased(df, 'Gender')
        if gender_biased:
            print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
            df = df.drop(columns='Gender')
        else:
            print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

    return trait_biased, df


In [ ]:
def judge_binary_variable_biased(dataframe, col_name, min_proportion=0.1, min_num=5):
    """
    Check if the distribution of a binary variable in the dataset is too biased to be usable for analysis
    :param dataframe:
    :param col_name:
    :param min_proportion:
    :param min_num:
    :return:
    """
    label_counter = dataframe[col_name].value_counts()
    total_samples = len(dataframe)
    rare_label_num = label_counter.min()
    rare_label = label_counter.idxmin()
    rare_label_proportion = rare_label_num / total_samples

    print(
        f"For the feature \'{col_name}\', the least common label is '{rare_label}' with {rare_label_num} occurrences. This represents {rare_label_proportion:.2%} of the dataset.")

    biased = (len(label_counter) < 2) or ((rare_label_proportion < min_proportion) and (rare_label_num < min_num))
    return bool(biased)

In [ ]:
def judge_continuous_variable_biased(dataframe, col_name):
    """Check if the distribution of a continuous variable in the dataset is too biased to be usable for analysis.
    As a starting point, we consider it biased if all values are the same. For the next step, maybe ask GPT to judge
    based on quartile statistics combined with its common sense knowledge about this feature.
    """
    quartiles = dataframe[col_name].quantile([0.25, 0.5, 0.75])
    min_value = dataframe[col_name].min()
    max_value = dataframe[col_name].max()

    # Printing quartile information
    print(f"Quartiles for '{col_name}':")
    print(f"  25%: {quartiles[0.25]}")
    print(f"  50% (Median): {quartiles[0.5]}")
    print(f"  75%: {quartiles[0.75]}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    biased = min_value == max_value

    return bool(biased)

In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Kidney Papillary Cell Carcinoma', the least common label is '0.0' with 20 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Kidney Papillary Cell Carcinoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 66.0
  75%: 67.0
Min: 48.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 8 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [ ]:
def save_cohort_info(cohort: str, info_path: str, is_available: bool, is_biased: Optional[bool] = None,
                     df: Optional[pd.DataFrame] = None, note: str = '') -> None:
    """
    Add or update information about the usability and quality of a dataset for statistical analysis.

    Parameters:
    cohort (str): A unique identifier for the dataset.
    info_path (str): File path to the JSON file where records are stored.
    is_available (bool): Indicates whether both the genetic data and trait data are available in the dataset, and can be
     preprocessed into a dataframe.
    is_biased (bool, optional): Indicates whether the dataset is too biased to be usable.
        Required if `is_available` is True.
    df (pandas.DataFrame, optional): The preprocessed dataset. Required if `is_available` is True.
    note (str, optional): Additional notes about the dataset.

    Returns:
    None: The function does not return a value but updates or creates a record in the specified JSON file.
    """
    if is_available:
        assert (df is not None) and (is_biased is not None), "'df' and 'is_biased' should be provided if this cohort " \
                                                             "is relevant."
    is_usable = is_available and (not is_biased)
    new_record = {"is_usable": is_usable,
                  "is_available": is_available,
                  "is_biased": is_biased if is_available else None,
                  "has_age": "Age" in df.columns if is_available else None,
                  "has_gender": "Gender" in df.columns if is_available else None,
                  "sample_size": len(df) if is_available else None,
                  "note": note}
    
    if not os.path.exists(info_path):
        with open(info_path, 'w') as file:
            json.dump({}, file)
        print(f"A new JSON file was created at: {info_path}")

    with open(info_path, "r") as file:
        records = json.load(file)
    records[cohort] = new_record

    temp_path = info_path + ".tmp"
    try:
        with open(temp_path, 'w') as file:
            json.dump(records, file)
        os.replace(temp_path, info_path)

    except Exception as e:
        print(f"An error occurred: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise

In [ ]:
import json
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
def read_json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a pandas DataFrame.

    Args:
    json_file (str): The path to the JSON file containing the data.

    Returns:
    DataFrame: A pandas DataFrame with the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'cohort_id'})

In [ ]:
def filter_and_rank_cohorts(json_file: str, condition: Union[str, None] = None) -> Tuple[
    Union[str, None], pd.DataFrame]:
    """
    Reads a JSON file, filters cohorts based on usability and an optional condition, then ranks them by sample size.

    Args:
    json_file (str): The path to the JSON file containing the data.
    condition (str, optional): An additional condition for filtering. If None, only 'is_usable' is considered.

    Returns:
    Tuple: A tuple containing the best cohort ID (str or None if no suitable cohort is found) and
           the filtered and ranked DataFrame.
    """
    # Read the JSON file into a DataFrame
    df = read_json_to_dataframe(json_file)

    if condition:
        filtered_df = df[(df['is_usable'] == True) & (df[condition] == True)]
    else:
        filtered_df = df[df['is_usable'] == True]

    ranked_df = filtered_df.sort_values(by='sample_size', ascending=False)
    best_cohort_id = ranked_df.iloc[0]['cohort_id'] if not ranked_df.empty else None

    return best_cohort_id, ranked_df


In [ ]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,320,


In [ ]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'Xena'

In [ ]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,320,


In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Kidney Papillary Cell Carcinoma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,53.0,1,0.895608,-2.059826,-0.531035,0.126528,-0.345878,1.12319,0.509706,...,-2.090786,0.932918,-0.150778,0.613183,2.243367,1.33611,0.226223,-1.415027,1.572405,0.879267
1,1,71.0,1,0.899708,-2.980426,0.049665,0.552628,-0.381678,0.17029,1.180806,...,-2.090786,-0.086682,0.565422,2.307883,0.294467,-1.28139,0.637223,-2.017627,2.026605,-0.408633
2,1,71.0,0,0.552108,-2.707826,-0.531035,0.473928,-0.255978,0.14389,-0.407894,...,2.040714,-0.086682,-0.748878,0.650683,-0.094533,-1.28139,-0.826577,-1.702427,1.751805,-0.769733
3,1,67.0,1,0.867508,-4.064326,-0.531035,-0.493572,-0.203878,0.83559,1.648106,...,-2.090786,-0.086682,-0.303278,-1.587117,-2.384733,2.91811,0.174223,-2.745727,2.166005,-2.483133
4,1,80.0,1,0.783608,-1.563626,0.829965,0.357528,-0.711678,0.34799,0.500906,...,-2.090786,-0.086682,-0.748878,1.423483,1.622367,0.46401,0.503323,-1.287227,1.909005,0.152467


In [ ]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
def detect_batch_effect(X):
    """
    Detect potential batch effects in a dataset using eigenvalues of XX^T.

    Args:
    X (numpy.ndarray): A feature matrix with shape (n_samples, n_features).

    Returns:
    bool: True if a potential batch effect is detected, False otherwise.
    """
    n_samples = X.shape[0]

    # Computing XX^T
    XXt = np.dot(X, X.T)

    # Compute the eigenvalues of XX^T
    eigen_values = np.linalg.eigvalsh(XXt)  # Using eigvalsh since XX^T is symmetric
    eigen_values = sorted(eigen_values, reverse=True)

    # Check for large gaps in the eigenvalues
    for i in range(len(eigen_values) - 1):
        gap = eigen_values[i] - eigen_values[i + 1]
        if gap > 1 / n_samples:  # You may need to adjust this threshold
            return True

    return False

In [ ]:
import numpy as np
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:
from sparse_lmm import VariableSelection


In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
def cross_validation(X, Y, Z, model_constructor, model_params, k=5, target_type='binary'):
    assert target_type in ['binary', 'continuous'], "The target type must be chosen from 'binary' or 'continuous'"
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    fold_size = len(X) // k
    performances = []

    for i in range(k):
        # Split data into train and test based on the current fold
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.setdiff1d(indices, test_indices)

        X_train, X_test = X[train_indices], X[test_indices]
        Y_train, Y_test = Y[train_indices], Y[test_indices]
        Z_train, Z_test = Z[train_indices], Z[test_indices]

        normalized_X_train, normalized_X_test = normalize_data(X_train, X_test)
        normalized_Z_train, normalized_Z_test = normalize_data(Z_train, Z_test)

        # model = model_constructor(**model_params)
        model = ResidualizationRegressor(model_constructor, model_params)
        model.fit(normalized_X_train, Y_train, normalized_Z_train)
        predictions = model.predict(normalized_X_test, normalized_Z_test)

        if target_type == 'binary':
            predictions = (predictions > 0.5).astype(int)
            Y_test = (Y_test > 0.5).astype(int)
            performance = accuracy_score(Y_test, predictions)
        elif target_type == 'continuous':
            performance = mean_squared_error(Y_test, predictions)

        performances.append(performance)

    cv_mean = np.mean(performances)
    cv_std = np.std(performances)

    if target_type == 'binary':
        print(f'The cross-validation accuracy is {(cv_mean * 100):.2f}% ± {(cv_std * 100):.2f}%')
    else:
        print(f'The cross-validation MSE is {(cv_mean * 100):.2f} ± {(cv_std * 100):.2f}')

    return cv_mean, cv_std

In [ ]:
def normalize_data(X_train, X_test=None):
    """Compute the mean and standard deviation statistics of the training data, use them to normalize the training data,
    and optionally the test data"""
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    # Handling columns with std = 0
    std_no_zero = np.where(std == 0, 1, std)

    # Normalize X_train
    X_train_normalized = (X_train - mean) / std_no_zero
    # Set normalized values to 0 where std was 0
    X_train_normalized[:, std == 0] = 0

    if X_test is not None:
        X_test_normalized = (X_test - mean) / std_no_zero
        X_test_normalized[:, std == 0] = 0
    else:
        X_test_normalized = None

    return X_train_normalized, X_test_normalized

In [ ]:
class ResidualizationRegressor:
    def __init__(self, regression_model_constructor, params=None):
        if params is None:
            params = {}
        self.regression_model = regression_model_constructor(**params)
        self.beta_Z = None  # Coefficients for regression of Y on Z
        self.beta_X = None  # Coefficients for regression of residual on X
        self.neg_log_p_values = None  # Negative logarithm of p-values
        self.p_values = None  # Actual p-values

    def _reshape_data(self, data):
        """
        Reshape the data to ensure it's in the correct format (2D array).

        :param data: The input data (can be 1D or 2D array).
        :return: Reshaped 2D array.
        """
        if data.ndim == 1:
            return data.reshape(-1, 1)
        return data

    def _reshape_output(self, data):
        """
        Reshape the output data to ensure it's in the correct format (1D array).

        :param data: The output data (can be 1D or 2D array).
        :return: Reshaped 1D array.
        """
        if data.ndim == 2 and data.shape[1] == 1:
            return data.ravel()
        return data
    
    def fit(self, X, Y, Z):
        X = self._reshape_data(X)
        Y = self._reshape_data(Y)
        Z = self._reshape_data(Z)

        # Step 1: Linear regression of Y on Z
        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        self.beta_Z = np.linalg.pinv(Z_ones.T @ Z_ones) @ Z_ones.T @ Y
        Y_hat = Z_ones @ self.beta_Z
        e_Y = Y - Y_hat  # Residual of Y

        # Step 2: Regress the residual on X using the included regression model
        self.regression_model.fit(X, e_Y)

        # Obtain coefficients from the regression model
        if hasattr(self.regression_model, 'coef_'):
            self.beta_X = self.regression_model.coef_
        elif hasattr(self.regression_model, 'getBeta'):
            beta_output = self.regression_model.getBeta()
            self.beta_X = self._reshape_output(beta_output)

        # Obtain negative logarithm of p-values, if available
        if hasattr(self.regression_model, 'getNegLogP'):
            neg_log_p_output = self.regression_model.getNegLogP()
            if neg_log_p_output is not None:
                self.neg_log_p_values = self._reshape_output(neg_log_p_output)
                self.p_values = np.exp(-self.neg_log_p_values)
                # Concatenate the p-values of Z and X. The p-values of Z were not computed, mark with NaN.
                p_values_Z = np.full(Z.shape[1], np.nan)
                self.p_values = np.concatenate((p_values_Z, self.p_values))
    def predict(self, X, Z):
        X = self._reshape_data(X)
        Z = self._reshape_data(Z)

        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        ZX = np.column_stack((Z, X))
        combined_beta = np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))
        return ZX @ combined_beta + self.beta_Z[0]

    def get_coefficients(self):
        return np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))

    def get_p_values(self):
        return self.p_values


In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error


In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 50.62% ± 6.30%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 93.75% ± 1.71%


In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 4. Discussion and report

In [ ]:
def interpret_result(model: Any, feature_names: List[str], trait: str, condition: str,
                     threshold: float = 0.05, save_output: bool = True,
                     output_dir: str = './output', model_id: int = 1) -> None:
    """This function interprets and reports the result of a trained linear regression model, where the regressor
    consists of one variable about condition and multiple variables about genetic factors.
    The function extracts coefficients and p-values from the model, and identifies the significant genes based on
    p-values or non-zero coefficients, depending on the availability of p-values.

    Parameters:
    model (Any): The trained regression Model.
    feature_names (List[str]): A list of feature names corresponding to the model's coefficients.
    trait (str): The target trait of interest.
    condition (str): The specific condition to examine within the model.
    threshold (float): Significance level for p-value correction. Defaults to 0.05.
    save_output (bool): Flag to determine whether to save the output to a file. Defaults to True.
    output_dir (str): Directory path where output files are saved. Defaults to './output'.
    model_id (int): The index of the model, 1 or 2.

    Returns:
    None: This function does not return anything but prints and optionally saves the output.
    """
    coefficients = model.get_coefficients().reshape(-1).tolist()
    p_values = model.get_p_values()
    if p_values is None:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients
        })
    else:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients,
            'p_value': p_values.reshape(-1).tolist()
        })
    condition_effect = regression_df[regression_df['Variable'] == condition].iloc[0]

    print(f"Effect of the condition on the target variable:")
    print(f"Variable: {condition}")
    print(f"Coefficient: {condition_effect['Coefficient']:.4f}")
    gene_regression_df = regression_df[regression_df['Variable'] != condition]
    if p_values is None:
        gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()
        significant_genes = gene_regression_df[gene_regression_df['Coefficient'] != 0]
        significant_genes_sorted = significant_genes.sort_values(by='Absolute Coefficient', ascending=False)
        print(
            f"Found {len(significant_genes_sorted)} genes with non-zero coefficients associated with the trait '{trait}' "
            f"conditional on the factor '{condition}'. These genes are identified as significant based on the regression model.")
    else:
        # Apply the Benjamini-Hochberg correction, to get the corrected p-values
        corrected_p_values = multipletests(gene_regression_df['p_value'], alpha=threshold, method='fdr_bh')[1]
        gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
        significant_genes = gene_regression_df.loc[gene_regression_df['corrected_p_value'] < threshold]
        significant_genes_sorted = significant_genes.sort_values('corrected_p_value')
        print(
            f"Found {len(significant_genes_sorted)} significant genes associated with the trait '{trait}' conditional on "
            f"the factor '{condition}', with corrected p-value < {threshold}:")

    print(significant_genes_sorted.to_string(index=False))

    # Optionally, save this to a CSV file
    if save_output:
        significant_genes_sorted.to_csv(
            os.path.join(output_dir, f'significant_genes_condition_{condition}_{model_id}.csv'), index=False)

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0037
Found 90 genes with non-zero coefficients associated with the trait 'Kidney Papillary Cell Carcinoma' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
     GAD2     2.675025              2.675025
     TSHR     2.231497              2.231497
    OR2H1    -2.160379              2.160379
     CHRD     1.936760              1.936760
  SNORA70     1.636720              1.636720
 SNORA59B     1.495033              1.495033
    MORC1     1.468713              1.468713
LOC284788     1.396849              1.396849
     MYL2     1.364502              1.364502
   OR7E5P     1.314911              1.314911
  MRGPRX2     1.095170              1.095170
   DHRS7C    -1.085291              1.085291
     NPPB     1.044539              1.044539
  TMEM41A     1.032128              1.032128
  PPIAL4E    -1.006413            

/var/folders/hx/qdnjfcxj2kbfy5k46ywrbrz40000gn/T/ipykernel_4404/2152198575.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [ ]:
from statsmodels.stats.multitest import multipletests


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0037
Found 128 significant genes associated with the trait 'Kidney Papillary Cell Carcinoma' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
   FRMD7    -0.145952 5.225444e-26       9.060398e-22
   TRPV5    -0.097127 6.315198e-24       5.474961e-20
   INSRR    -0.130892 8.222202e-23       4.752159e-19
ATP6V1G3    -0.135128 1.376995e-18       5.968930e-15
 TMEM207    -0.111532 1.883534e-17       6.531719e-14
  SLC4A9    -0.109657 2.003581e-15       5.790014e-12
HEPACAM2    -0.108074 3.576631e-14       8.859315e-11
   SEMG2    -0.059084 2.095954e-13       4.542717e-10
 FAM184B    -0.085460 6.844066e-13       1.318547e-09
    ELF5    -0.078186 3.046500e-12       5.282326e-09
   OXGR1    -0.088583 4.930038e-12       7.771084e-09
C10orf71    -0.061142 1.210500e-11       1.749072e-08
    HMX2    -0.077829 2.403095e-11       3.205175e-08
  NKX6-2 

/var/folders/hx/qdnjfcxj2kbfy5k46ywrbrz40000gn/T/ipykernel_4404/2152198575.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
